In [2]:
#!pip install pypdf

In [4]:
from langchain.agents import Tool, AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

import os
from dotenv import load_dotenv

# 2️⃣ Load API keys
load_dotenv(".env")

True

In [5]:
loader = PyPDFLoader("MOHAN J_Resume_Updated.pdf")
documents = loader.load()


In [6]:
documents

[Document(metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '2025-11-24T17:15:38+05:30', 'moddate': '2025-11-25T04:56:59+00:00', 'source': 'MOHAN J_Resume_Updated.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='MOHAN J \n   Phone: +91-9535881071 |         Email: jmohan2787@gmail.com \n   LinkedIn: https://www.linkedin.com/in/mohan-j-783b46105/ \nGitHub:    https://github.com/jmohan2787?tab=repositories \n \nCAREER SUMMARY: \nExperienced Telecommunications Engineer with 16+ years in optical networking, hardware \ntesting, and Python automation. Expertise includes: \n• 6+ years in Hardware Testing & Debugging for Optical Products \n• 5+ years in SDH/SONET Manual Hardware Testing \n• 4+ years in Python Automation for NOKIA 1830PSS DWDM Products \nProficient in test case creation, automation scripting, troubleshooting, and debugging for Nokia, \nTejas, and multi-vendor optical transport systems. Adept at working with JIRA, APT \nFramework, SPLUNK, a

In [7]:
llm = ChatOpenAI(temperature=0)

C:\Users\mojayara\AppData\Local\Temp\ipykernel_26600\3681847723.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0)


In [16]:
splitter = CharacterTextSplitter(separator="\n", chunk_size=300, chunk_overlap=50)
texts = splitter.split_documents(documents)
embedding = OpenAIEmbeddings()
# vectorstore = Chroma.from_texts(texts, embedding)
vectorstore = Chroma.from_documents(texts, embedding)
retriever = vectorstore.as_retriever()

In [22]:
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000154AD048520>, search_kwargs={})

In [17]:
# 5️⃣ Create RetrievalQA Chain
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# 6️⃣ Convert to LangChain Tool
qa_tool = Tool(
    name="LangChainRetriever",
    func=qa_chain.run,
    description="Use this to answer questions about LangChain framework, features, or its creator."
)

# 7️⃣ Setup Memory (to support conversational context)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 8️⃣ Initialize Agent with Tool + Memory
agent = initialize_agent(
    tools=[qa_tool],
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,  # ReAct agent
    memory=memory,
    verbose=True,
    handle_parsing_errors=True
)



C:\Users\mojayara\AppData\Local\Temp\ipykernel_26600\1353073768.py:12: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
C:\Users\mojayara\AppData\Local\Temp\ipykernel_26600\1353073768.py:15: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent 

In [23]:
# # 9️⃣ Ask Questions (RAG-Style)
# print("1️⃣ First Question")
# res1 = agent.run("May i know what is this pdf about?")
# print("Answer:", res1)

# print("\n2️⃣ Follow-up")
# res2 = agent.run("Can you summrize experiance in this file?")
# print("Answer:", res2)

print("\n3️⃣ Combined Reasoning")
res3 = agent.run("May i know list of company he worked?")
print("Answer:", res3)


3️⃣ Combined Reasoning


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: LangChainRetriever
Action Input: list of companies the individual worked for
Observation: The individual worked for two companies:
1. Infinite Computer Solutions in Chennai from September 2015 to May 2017 as a Senior Software Engineer.
2. The individual also worked as a Senior Technical Lead Engineer, but the company name is not provided in the context.
Thought:Do I need to use a tool? No
AI: The individual worked for two companies:
1. Infinite Computer Solutions in Chennai from September 2015 to May 2017 as a Senior Software Engineer.
2. The individual also worked as a Senior Technical Lead Engineer, but the company name is not provided in the context.

> Finished chain.
Answer: The individual worked for two companies:
1. Infinite Computer Solutions in Chennai from September 2015 to May 2017 as a Senior Software Engineer.
2. The individual also worked as a Senior Technical Lea